# GENet : GPU Efficient Network

![](https://raw.githubusercontent.com/idstcv/GPU-Efficient-Networks/master/misc/genet_acc_speed_curve.jpg)

The proposed design space is optimized for fast GPU inference. In this space, it uses a semi-automatic NAS to
help us design GPU-Efficient Networks. GENets use full convolutions in low-level stages and depth-wise
convolution and/or bottleneck structure in high-level stages. 

This design is inspired by the observation that convolutional kernels in the high-level stages are more likely to have low intrinsic rank and different types of convolutions have different kinds of efficiency on GPU.

# Albumentations
Albumentations is a Python library for image augmentation. Image augmentation is used in deep learning and computer vision tasks to increase the quality of trained models. The purpose of image augmentation is to create new training samples from the existing data.

* Albumentations supports all common computer vision tasks such as classification, semantic segmentation, instance segmentation, object detection, and pose estimation.
* The library provides a simple unified API to work with all data types: images (RBG-images, grayscale images, multispectral images), segmentation masks, bounding boxes, and keypoints.
* The library contains more than 70 different augmentations to generate new training samples from the existing data.
* Albumentations is fast.

* Installation:- pip install -U albumentations

# Load GENets

In [ ]:
!git clone https://github.com/idstcv/GPU-Efficient-Networks.git

In [ ]:
cd ./GPU-Efficient-Networks

In [ ]:
import GENet

In [ ]:
cd ../

# Import Libraries

In [ ]:
# Python library to interact with the file system.
import os

# Software library written for data manipulation and analysis.
import pandas as pd

# fastai library for computer vision tasks
from fastai.vision.all import *

# Python library for image augmentation
import albumentations as A


# Load training data

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df

In [ ]:
train_df['image_id'] = train_df['image_id'].map(lambda x : path /'train_images'/x )
train_df.head()

# Create Dataloaders

In [ ]:
# obtain the input images.
def get_x(r):
    return r['image_id']

# obtain the targets.
def get_y(r):
    return r['label']

The Albumentation code has been borrowed from Fastai [docs](https://docs.fast.ai/tutorial.albumentations.html). It's very common to use different transforms on the training dataset versus the validation dataset. Lets see how!

In [ ]:
'''AlbumentationsTransform will perform different transforms over both
   the training and validation datasets ''' 
class AlbumentationsTransform(RandTransform):
    
    '''split_idx is None, which allows for us to say when we're setting our split_idx.
       We set an order to 2 which means any resize operations are done first before our new transform. '''
    split_idx, order = None, 2
    
    def __init__(self, train_aug, valid_aug): store_attr()
    
    # Inherit from RandTransform, allows for us to set that split_idx in our before_call.
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    # If split_idx is 0, run the trainining augmentation, otherwise run the validation augmentation. 
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [ ]:
def get_train_aug(size): 
    
    return A.Compose([
            # allows to combine RandomCrop and RandomScale
            A.RandomResizedCrop(size,size),
            
            # Transpose the input by swapping rows and columns.
            A.Transpose(p=0.5),
        
            # Flip the input horizontally around the y-axis.
            A.HorizontalFlip(p=0.5),
        
            # Flip the input horizontally around the x-axis.
            A.VerticalFlip(p=0.5),
        
            # Randomly apply affine transforms: translate, scale and rotate the input.
            A.ShiftScaleRotate(p=0.5),
        
            # Randomly change hue, saturation and value of the input image.
            A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        
            # Randomly change brightness and contrast of the input image.
            A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        
            # CoarseDropout of the rectangular regions in the image.
            A.CoarseDropout(p=0.5),
        
            # CoarseDropout of the square regions in the image.
            A.Cutout(p=0.5) ])

def get_valid_aug(size): 
    
    return A.Compose([
    # Crop the central part of the input.   
    A.CenterCrop(size, size, p=1.),
    
    # Resize the input to the given height and width.    
    A.Resize(size,size)], p=1.)

In [ ]:
'''The first step item_tfms resizes all the images to the same size (this happens on the CPU) 
   and then batch_tfms happens on the GPU for the entire batch of images. '''
# Transforms we need to do for each image in the dataset
item_tfms = [Resize(256), AlbumentationsTransform(get_train_aug(256), get_valid_aug(256))]

# Transforms that can take place on a batch of images
batch_tfms = [Normalize.from_stats(*imagenet_stats)]

In [ ]:
def get_data(bs=32, data_df=train_df):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter=RandomSplitter(seed=42), # split data into training and validation subsets.
                       get_x=get_x, # obtain the input images.
                       get_y=get_y, # obtain the targets.
                       item_tfms = item_tfms,
                       batch_tfms = batch_tfms)
    return dblock.dataloaders(data_df,bs=bs)

dls = get_data()

In [ ]:
# We can call show_batch() to see what a sample of a batch looks like.
dls.show_batch()

# Model Definition

* There are three pre-trained models, GENet-large/normal/small.
* GENet_large (31M param)
* GENet_normal (21M param)
* GENet_small (8.17M)

    * GENet-large/normal/small use different input image resolutions.
    * GENet-large, size = 256
    * GENet-normal, size = 192
    * GENet-small, size = 192
    

In [ ]:
model = GENet.genet_large(pretrained=True, root='../input/genetparam/')

In [ ]:
# Group together some dls, a model, and metrics to handle training
learn = Learner(dls, model, metrics = accuracy) 

In [ ]:
# Choosing a good learning rate
learn.lr_find()

In [ ]:
# We can use the fine_tune function to train a model with this given learning rate
learn.fine_tune(4, base_lr=0.0012022644514217973)

### 85% accuracy in 5 epochs! Not Bad!!

In [ ]:
# Plot training and validation losses.
learn.recorder.plot_loss()

In [ ]:
# Interpretation methods for classification models.
interp = ClassificationInterpretation.from_learner(learn)

# Show images in top_losses along with their prediction, actual, loss, and probability of actual class.
interp.plot_top_losses(5, nrows=5)

# Make Submission file

In [ ]:
sample = pd.read_csv(path/'sample_submission.csv')
sample

In [ ]:
_sample = sample.copy()
_sample['image_id'] = _sample['image_id'].map(lambda x:path/'test_images'/x)
test_dl = dls.test_dl(_sample)

In [ ]:
_sample.head()

In [ ]:
test_dl.show_batch()

# Test Time Augmentation (TTA)
Similar to what Data Augmentation is doing to the training set, the purpose of Test Time Augmentation is to perform random modifications to the test images. Thus, instead of showing the regular, “clean” images, only once to the trained model, we will show it the augmented images several times. We will then average the predictions of each corresponding image and take that as our final guess.

The reason why it works is that, by averaging our predictions, on randomly modified images, we are also averaging the errors. The error can be big in a single vector, leading to a wrong answer, but when averaged, only the correct answer stand out.

In [ ]:
a, _ = learn.tta(dl=test_dl, n=8)
pred = a.argmax(dim=1).numpy()
sample['label'] = pred

In [ ]:
sample.to_csv('submission.csv',index=False)

## *Upvote the kernel if you found it insightful!*